# <span style="color: blue;">Области видимости</span>

В отличие от Java (< 8), C/C++ (< 11) в Python функции -- это объекты первого класса (полноправные значения)

Поэтому с ними можно делать всё то же самое, что и с другими значениями.

Например, можно объявлять функции внутри других функций:

In [ ]:
def wrapper():
    def identity(x):
        return x
    return identity

f = wrapper()
f(42)  # 42

In [ ]:
def wrapper(func):
    
    def wrapped(*args, **kwargs):
        print('before')
        res = func(*args, **kwargs)
        print('after')
        return res
    
    return wrapped

def fun():
    print('fun')
    
fun()

In [ ]:
fun = wrapper(fun)
fun()

In [ ]:
# фабрика функций min
def make_min(*, lo, hi):
    def inner(first, *args):
        res = hi
        for arg in (first, ) + args:
            if lo <= arg < res:
                res = arg
        return res
    return inner

bounded_min = make_min(lo=0, hi=255)
bounded_min(-5, 12, 13)

## Области видимости LEGB

In [ ]:
min  # builtin (встроенная)
min = 42  # global (глобальная)
def f(*args):
    min = 2
    def g():  # enclosing (объемлющая)
        min = 4  # local (локальная)
        print(min)

`enclosing` -- это как бы "родительская" область видимости той функции, в которой объявлена наша функция

Этой области может и не быть

Внутри функции `g()` переменная `min=2` — находится в объемлющей области видимости

Поиск имени ведётся в четырёх областях: L, E, G, B 

### Интроспеция

In [ ]:
min = 42  # = globals()["min"] = 42
globals()
# {..., 'min': 42}

def f():
    min = 2  # = locals()["min"] = 2
    print(locals())
f()

Они даже изменяемые, но лучше так не менять :)

In [ ]:
globals()["maxxx"] = 123
maxxx

## Замыкания
Функции в Python могут использовать переменные, определённые во внешних областях видимости

Важно: Поиск переменных осуществляется во время исполнения функции, а не во время её объявления

In [ ]:
def f():
    print(i)
    
for i in range(4):
    f()

**Замыкание** -- функция, которая ссылается на свободные переменные в своём лексическом контексте

In [ ]:
def make_adder(x):
    def adder(n):
        return x + n  # захват `x` из внешнего контекста
    return adder

adder = make_adder(10)
adder(5)

In [ ]:
adder2 = make_adder(20)
adder2(5)

In [ ]:
adder(5)

## Присваивание

Для присваивания правило LEGB не работает

In [ ]:
min = 42
def f():
    min += 1
    # min = min   -- тоже даст ошибку
    return min
f()

#### Почему так?

Это разворачивается следующим образом: `min = min + 1`

По умолчанию операция присваивания создаёт локальную переменную.
<br/>
Поэтому слева создаётся локальная переменная, а справа используется глобальная.

Считается, что тут скорее всего ошибка, неявное поведение.

Изменить это поведение можно с помощью операторов **`global`** и **`nonlocal`**

## Оператор `global`

In [ ]:
min = 42
def f():
    global min
    min += 1
    return min

In [ ]:
f()

In [ ]:
f()

Но использование **`global`** порочно _(лучше его вообще не использовать)_

## Оператор `nonlocal`

Для модификации значения переменной из объемлющей области видимости

In [ ]:
def cell(value=None):
    def get():
        return value
    def set(update):
        nonlocal value
        value = update
    return get, set

get, set = cell()
set(42)
get()

При вызове функции создаётся новая область видимости

Т.е. другие вызовы `cell()` породили бы другую переменную `value`

In [ ]:
del cell  # удалит функцию из глобальной области видимости, но get и set будут работать :)
set(24)
get()

Подробнее: http://python.org/dev/peps/pep-3104